In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

os.listdir('../input/tcd-ml-comp-201920-rec-alg-click-pred-group')

['tcdml1920-rec-click-pred--submission file (example).csv',
 'tcdml1920-rec-click-pred--submission file.csv',
 'tcdml1920-rec-click-pred--test.csv',
 'tcdml1920-rec-click-pred--training.csv']

In [2]:
import pickle

train = pickle.load(open('train.pkl', 'rb'))
test = pickle.load(open('test.pkl', 'rb'))

In [12]:
train.head()

,recommendation_set_id,query_word_count,query_char_count,query_detected_language,organization_id,item_type,hour_request_received,app_version,app_lang,country_by_ip,local_hour_of_request,recommendation_algorithm_id_used,algorithm_class,cbf_parser,search_title,search_keywords,search_abstract,set_clicked
0,46897,6,35,en,1,academic_publication,16,4.3.1,en,NL,15,1,content_based_filtering,standard_QP,True,False,False,0
1,46898,9,53,en,1,academic_publication,16,4.3.1,en,GB,15,1,content_based_filtering,standard_QP,True,False,False,0
2,46899,17,123,en,1,academic_publication,16,4.3.1,en,JP,15,2,content_based_filtering,edismax_QP,True,True,False,0
3,46902,14,100,en,1,academic_publication,16,4.3.1,en,US,15,1,content_based_filtering,standard_QP,True,False,False,0
4,46903,14,100,en,1,academic_publication,16,4.3.1,en,US,15,2,content_based_filtering,edismax_QP,True,True,False,0


### Scaling and Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

all_data = pd.concat((train.loc[:, 'query_word_count': 'search_abstract'],
                      test.loc[:, 'query_word_count': 'search_abstract']))

# Scaling
numerical_features = all_data.select_dtypes(exclude = ["object"]).columns
stdSc = StandardScaler()
all_data.loc[:, numerical_features] = stdSc.fit_transform(all_data.loc[:, numerical_features])

# Encoding
le = LabelEncoder()

cols_to_label_encode = ['query_detected_language', 'item_type', 'app_version', 'app_lang', 'country_by_ip']
for col in cols_to_label_encode:
    all_data[col] = le.fit_transform(all_data[col])

all_data = pd.get_dummies(all_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [14]:
all_data.head()

,app_lang,app_version,country_by_ip,hour_request_received,item_type,local_hour_of_request,organization_id,query_char_count,query_detected_language,query_word_count,...,algorithm_class_sentence_embeddings,algorithm_class_stereotype,algorithm_class_unknown,application_type_0,application_type_blog,application_type_digital_library,application_type_e-commerce,cbf_parser_edismax_QP,cbf_parser_mlt_QP,cbf_parser_standard_QP
0,3,50,129,0.515563,10,0.166288,-0.597329,-1.027837,8,-0.762760,...,0,0,0,0,0,0,0,0,0,1
1,3,50,57,0.515563,10,0.166288,-0.597329,-0.469985,8,-0.128165,...,0,0,0,0,0,0,0,0,0,1
2,3,50,87,0.515563,10,0.166288,-0.597329,1.699438,8,1.564089,...,0,0,0,0,0,0,0,1,0,0
3,3,50,175,0.515563,10,0.166288,-0.597329,0.986627,8,0.929494,...,0,0,0,0,0,0,0,0,0,1
4,3,50,175,0.515563,10,0.166288,-0.597329,0.986627,8,0.929494,...,0,0,0,0,0,0,0,1,0,0


## Train test split

In [15]:
from sklearn.model_selection import train_test_split

final_test = all_data[train.shape[0]:]

X = all_data[:train.shape[0]]
y = train['set_clicked'].astype(bool)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Models

## Random Forrest

In [16]:
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# To beat 0.015915

for n_est in [5, 10, 20, 50, 100]:
    RF = RandomForestClassifier(n_estimators=n_est ,n_jobs=-1)
    RF.fit(X_train, y_train)
    round(RF.score(X_train, y_train), 4)

    predicted_labels = RF.predict(X_test)
    
    print("Accuracy Score: ", accuracy_score(y_test, predicted_labels))
    print("F1 Score: ", f1_score(list(y_test), predicted_labels))

Accuracy Score:  0.9803178722024003
F1 Score:  0.013012361743656473
Accuracy Score:  0.9806292572169965
F1 Score:  0.014521452145214523
Accuracy Score:  0.980525462212131
F1 Score:  0.014445173998686804
Accuracy Score:  0.9806292572169965
F1 Score:  0.014521452145214523


## LGB

In [17]:
import lightgbm as lgb

d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

clf = lgb.train(params, d_train, 100)